<a href="https://colab.research.google.com/github/Namtk214/Application-of-RAG-in-asking-for-documents/blob/main/LLM_MODEL_RAG_WITHOUT_UL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.25.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer , AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface . llms import HuggingFacePipeline

from langchain . memory import ConversationBufferMemory
from langchain_community . chat_message_histories import ChatMessageHistory
from langchain_community . document_loaders import PyPDFLoader, TextLoader
from langchain . chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core . runnables import RunnablePassthrough
from langchain_core . output_parsers import StrOutputParser
from langchain import hub

In [8]:
!pip install PyMuPDF
import fitz

def load_pdf(file_path):
    try:
        document = fitz.open(file_path)
        documents = []
        for page_num in range(len(document)):
            page = document[page_num]
            text = page.get_text()
            documents.append(text)
        document.close()
        return documents
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

FILE_PATH = '/content/YOLOv10_Tutorials.pdf'
documents = load_pdf(FILE_PATH)

# Print the content for verification
for i, doc in enumerate(documents, start=1):
    print(f"Page {i}:")
    print(doc)


Page 1:
AI VIET NAM – AI COURSE 2024
Tutorial: Phát hiện đối tượng trong ảnh với
YOLOv10
Dinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và
Quang-Vinh Dinh
Ngày 20 tháng 6 năm 2024
I.
Giới thiệu
Object Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổđiển thuộc lĩnh vực
Computer Vision. Mục tiêu của bài toán này là tựđộng xác định vịtrí của các đối tượng trong
một tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm
giải quyết hiệu quảbài toán này. Trong đó, các phương pháp thuộc họYOLO (You Only Look
Once) thu hút được sựchú ý rất lớn từcộng đồng nghiên cứu bởi độchính xác và tốc độthực
thi mà loại mô hình này mang lại.
Hình 1: Logo của mô hình YOLO. Ảnh: link.
Thời gian vừa qua, Ao Wang và các cộng sựtại Đại học Thanh Hoa (Tsinghua University)
đã đềxuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object
Detection [10]. Với những cải tiến mới, mô hình đã đạt được hiệu suất vượt trội hơn so với các
phiên bản YOLO trước đó 

In [9]:
text_splitter = RecursiveCharacterTextSplitter (chunk_size =1000, chunk_overlap =100)

In [10]:
# Convert the list of strings to Langchain Document objects
from langchain.schema import Document

docs = [Document(page_content=text) for text in documents]

# Now you can split the documents
docs = text_splitter.split_documents(docs)

print ("Number of sub - documents: ", len(docs))
print(docs[0])

Number of sub - documents:  32
page_content='AI VIET NAM – AI COURSE 2024\nTutorial: Phát hiện đối tượng trong ảnh với\nYOLOv10\nDinh-Thang Duong, Nguyen-Thuan Duong, Minh-Duc Bui và\nQuang-Vinh Dinh\nNgày 20 tháng 6 năm 2024\nI.\nGiới thiệu\nObject Detection (Tạm dịch: Phát hiện đối tượng) là một bài toán cổđiển thuộc lĩnh vực\nComputer Vision. Mục tiêu của bài toán này là tựđộng xác định vịtrí của các đối tượng trong\nmột tấm ảnh. Tính tới thời điểm hiện tại, đã có rất nhiều phương pháp được phát triển nhằm\ngiải quyết hiệu quảbài toán này. Trong đó, các phương pháp thuộc họYOLO (You Only Look\nOnce) thu hút được sựchú ý rất lớn từcộng đồng nghiên cứu bởi độchính xác và tốc độthực\nthi mà loại mô hình này mang lại.\nHình 1: Logo của mô hình YOLO. Ảnh: link.\nThời gian vừa qua, Ao Wang và các cộng sựtại Đại học Thanh Hoa (Tsinghua University)\nđã đềxuất mô hình YOLOv10 trong bài báo YOLOv10: Real-Time End-to-End Object\nDetection [10]. Với những cải tiến mới, mô hình đã đạt được hiệu 

In [11]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
vector_db = Chroma.from_documents(documents =docs, embedding = embedding)

retriever = vector_db.as_retriever()

In [14]:
result = retriever.invoke(" What is YOLO ?")
print(" Number of relevant documents : ", len(result))

 Number of relevant documents :  4


In [15]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit =True ,
    bnb_4bit_quant_type ="nf4",
    bnb_4bit_use_double_quant =True ,
    bnb_4bit_compute_dtype = torch.bfloat16
)

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "lmsys/vicuna-7b-v1.5"

nf4_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [17]:

model_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline
)


In [18]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

USER_QUESTION = "Object detection với YOLOv10"
output = rag_chain.invoke(USER_QUESTION)
answer = output.split('Answer:')[1].strip()
print(answer)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


YOLOv1 is a one-stage object detection model introduced in 2016. It uses a single neural network to predict the position and class of objects directly from the input image. It has the advantage of being fast and accurate for detecting objects in real-time. However, it has the disadvantage of not being very accurate for small objects or those that are occluded.
